# Regresja logistyczna

Model regresji logistycznej w R buduje się za pomocą funkcji `glm()`, która służy do dopasowania uogólnionych modeli liniowych. Funkcja ta wyznacza m.in. oszacowania współczynników `b` oraz wylicza wartości reszt. W celu budowy modelu regresji logistycznej należy przy wywołaniu funkcji `glm()` podać argument `family = "binomial"` (rodzina rozkładów dwumianowych).

Jako przykład wywołania funkcji `glm()` do budowy modelu regresji logistycznej wykorzystamy dane na temat cen domów z wykładu o regresji liniowej i wielorakiej (plik House-Price.csv). Aby samodzielnie przeanalizować przykład w RStudio, należy je najpierw poddać wstępnemu przetwarzaniu, podobnie jak na wykładzie). W najprostszym przypadku użyjemy tylko jednego predyktora: ceny (zmienna `price`) do przewidzenia czy dom zostanie sprzedany w przeciągu trzech miesięcy (zmienna `Sold`. Wartość 1 oznacza, że dom został sprzedany, 0 - nie sprzedany).

In [ ]:
library(readr)
df <- read_csv("./dane/House-Price.csv")
df$avg_dist = (df$dist1+df$dist2+df$dist3+df$dist4)/4

df2 <- df[,-6:-9]
df <- df2
rm(df2)

df <-df[,-13]

In [ ]:
glm.fit = glm(Sold~price, data = df, family = binomial)

summary(glm.fit)

W wyświetlonym podsumowaniu zostają przytoczone oceny współczynników dla kolejnych zmiennych objaśniających (w tym przypadku jedna zmienna `price`), odchylenia standardowe tych ocen, wartości statystyki testowej dla testu, w którym hipotezą zerową jest nieistotność danej zmiennej oraz p-wartość dla tego testu. Im p-wartość jest niższa, tym istotniejsza jest zmienna. Graficznie ta istotność jest przedstawiona gwiazdkami po prawej stronie. Wiersze tabeli odpowiadają zmiennym objaśniającym i wyrazowi wolnemu dopasowanego modelu.

Możemy odwołać się do konkretnych właściwości utworzonego obiektu klasy `glm`: - tabela oszacowań współczynników modelu (dodatkowo funkcja `$coef` lub `$coefficients` na obiekcie `summary`):

In [ ]:
glm.fit$coefficients

summary(glm.fit)$coef

-   dopasowane przez model prawdopodobieństwa "sukcesu" dla każdej z prób - wartości dopasowane przez model przekształcone przez funkcję wiążącą:

In [ ]:
glm.fit$fitted.values

-   reszty dla każdej z prób:

In [ ]:
glm.fit$residuals

-   rodzina rozkładów zmiennej objaśnianej oraz użyta funkcja wiążąca:

In [ ]:
glm.fit$family

-   liczba wykonanych iteracji algorytmu:

In [ ]:
glm.fit$iter

-   czy algorytm zakończył iterowanie bo znalazł maksimum (wartość `TRUE`) czy dlatego że wykonał więcej iteracji niż założono (wartość `FALSE`):

In [ ]:
glm.fit$converged

## Regresja logistyczna dla wielu zmiennych objaśniających

In [ ]:

glm.fit = glm(Sold~., data = df, family = binomial)

summary(glm.fit)

Na podstawie uzyskanych wyników możemy zbudować model końcowy (złożony ze zmiennych `price`, `air_qual`, `room_num`, `teachers`, `poor_prop`, `n_hos_beds` oraz `av_dist`). W celu wybrania najlepszego modelu możemy też skorzystać z funkcji `step()`.

In [ ]:
model_koncowy <- glm(formula = Sold~price+air_qual+room_num+teachers+poor_prop+
                       n_hos_beds+avg_dist, data = df, family = binomial)
summary(model_koncowy)

step(model_koncowy, direction = "backward")

Funkcja `vif` (z pakietu `car`) służy do zbadania współliniowości zmiennych:

In [ ]:
library(car)
vif(model_koncowy)

Analizę końcowego modelu możemy wykonać za pomocą szans (ilorazu prawdopodobieństwa sukcesu do prawdopodobieństwa porażki)

In [ ]:
exp(coef(model_koncowy))

Szansa dla zmiennej `price` wynosi 0.76, co oznacza, że wraz ze wzrostem ceny o jedną jednostkę szansa na sprzedaż domu maleje o ok 24%. Natomiast wraz ze zwiększeniem się ilości nauczycieli przypadających na tysiąc mieszkańców o jeden (zmienna `teachers`), szansa sprzedania domu w tej okolicy wzrasta średnio o 35%.

## Zadania

1.  Wczytaj dane `lab4-dane.csv`. Zawierają one informacje na temat matek i ich dzieci, u których wykryto pewną rzadko występującą wadę wrodzoną (grupa badana) oraz zdrowych (grupa kontrolna).

In [ ]:
# 

2.  Przeanalizuj dane oraz wyświetl podstawowe statystyki opisowe dla analizowanych zmiennych. Następnie narysuj histogramy dla zmiennych `Wiek` oraz `MasaUr`. Oceń rozkłady tych zmiennych.

In [ ]:
#

3.  Utwórz model regresji logistycznej by sprawdzić, które zmienne mogą wywierać istotny wpływ na występowanie wady. Następnie wybierz zmienne wpływowe i na ich podstawie utwórz model końcowy.

In [ ]:
#

4.  Oblicz współczynnik VIF dla modelu końcowego i sprawdź, czy występuje współliniowość w danych (współliniowość możemy też analizować za pomocą wykresu korelacji)

In [ ]:
#

5.  Wykonaj analizę modelu końcowego za pomocą szans.

In [ ]:
#